In [1]:
import ferm_utils as feru
import csa_utils as csau
import openfermion as of
import numpy as np
from sdstate import *
import random
import h5py
import sys
from matrix_utils import construct_orthogonal
import pickle

ps_path = "planted_solutions/"
# File name in ps_path folder
file_name = "2_co2_6-311++G___12_9d464efb-b312-45f8-b0ba-8c42663059dc.pkl"

def construct_Hamiltonian_with_solution(path, file_name, key = ""):
    """
    Load the planted_soluiton Hamiltonian from the planted_solution, with key to be the block structure of
    encoding the Hamiltonian.
    """
    with open(ps_path + file_name, 'rb') as handle:
        dic = pickle.load(handle)
    if not key:
        key = list(dic.keys())[0]
    cas_x = dic[key]["cas_x"]
    killer = dic[key]["killer"]
    k = dic[key]["k"]
    upnum = dic[key]["upnum"]
    spin_orbs = dic[key]["spin_orbs"]
#     Number of electrons in each CAS block
    e_nums = dic[key]["e_nums"]
#     Solution in a list of sdstates, each sdstate represent the ground state with in each CAS block.
# This can be generalized into the general solution with concatenate() in sdstates, but not recommended as it 
# requires exponential time and space.
    sol = dic[key]["sol"]
#     CAS 2e tensor
    tbt = csau.get_cas_matrix(cas_x, spin_orbs, k)
#     Construct hidden 2e tensor and Hamiltonian
    killer_tbt = feru.get_chemist_tbt(killer, spin_orbs, spin_orb = True)
    killer_one_body = of.normal_ordered(killer - feru.get_ferm_op(killer_tbt, spin_orb=True))
    killer_onebody_matrix = feru.get_obt(killer_one_body, n = spin_orbs, spin_orb = True)
    killer_onebody_tbt = feru.onebody_to_twobody(killer_onebody_matrix)
    Htbt_killer = np.add(killer_tbt, killer_onebody_tbt) 
    Htbt_with_killer = np.add(tbt, Htbt_killer)
#     Set up random unitary to hide 2e tensor, the parameters here are initally set as random unitary rotations
    random_uparams = np.random.rand(upnum)
    U = construct_orthogonal(spin_orbs, random_uparams)
#     Hide 2e etensor with random unitary transformation
    Htbt_hidden = csau.cartan_orbtransf(Htbt_with_killer, U, complex = False)
#     Hidden 2e tensor without killer
    tbt_hidden = csau.cartan_orbtransf(tbt, U, complex = False)
    return tbt, tbt_hidden, Htbt_with_killer, Htbt_hidden, sol, e_nums

tbt, tbt_hidden, Htbt_with_killer, Htbt_hidden, sol, e_nums = construct_Hamiltonian_with_solution(ps_path, file_name)

In [2]:
print(tbt.shape)

(12, 12, 12, 12)
